In [1]:
# change dir to ../
import os
import sys
# print the current working directory
os.chdir("..")
sys.path.append("src/cmx")

In [2]:
# function definition
import cv2
import numpy as np

def extract_blobs(segmentation_layer, size_threshold=100, merge_distance=10):
    """
    Extracts blobs from a binary segmentation layer, filters small blobs,
    and merges closely overlapping blobs.

    Args:
        segmentation_layer: Binary mask of the segmentation layer.
        size_threshold: Minimum area of a blob to be considered valid.
        merge_distance: Maximum distance between blobs to merge them.

    Returns:
        blobs: A list of dictionaries containing blob information.
    """
    # Find contours in the binary mask
    contours, _ = cv2.findContours(segmentation_layer, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a list to store filtered and merged blobs
    blobs = []

    for contour in contours:
        # Compute bounding box
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        
        # Filter out small blobs
        if area < size_threshold:
            continue
        
        # Compute centroid
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = x + w // 2, y + h // 2  # Fallback to bounding box center
        
        # Store blob information
        blobs.append({
            "contour": contour,
            "bounding_box": (x, y, w, h),
            "centroid": (cx, cy),
            "area": area
        })
    
    # Merge blobs that are close to each other
    merged_blobs = []
    used = [False] * len(blobs)
    
    for i, blob in enumerate(blobs):
        if used[i]:
            continue
        x1, y1, w1, h1 = blob["bounding_box"]
        cx1, cy1 = blob["centroid"]
        merged_contour = blob["contour"]
        
        for j, other_blob in enumerate(blobs):
            if i == j or used[j]:
                continue
            x2, y2, w2, h2 = other_blob["bounding_box"]
            cx2, cy2 = other_blob["centroid"]
            
            # Check if the blobs are within merge_distance
            if abs(cx1 - cx2) <= merge_distance and abs(cy1 - cy2) <= merge_distance:
                x1 = min(x1, x2)
                y1 = min(y1, y2)
                w1 = max(x1 + w1, x2 + w2) - x1
                h1 = max(y1 + h1, y2 + h2) - y1
                merged_contour = np.vstack((merged_contour, other_blob["contour"]))
                used[j] = True
        
        # Add merged blob to the final list
        merged_blobs.append({
            "contour": merged_contour,
            "bounding_box": (x1, y1, w1, h1),
            "centroid": (int(x1 + w1 / 2), int(y1 + h1 / 2)),
            "area": cv2.contourArea(merged_contour)
        })
        used[i] = True
    
    return merged_blobs

def separate_layers(mask):
    """
    Separates a mask image into individual layers depending on the pixel values.
    """
    layers = []
    for i in range(1, 5):
        layers.append((mask == i).astype(np.uint8))
    return np.asarray(layers)

In [3]:
# open a folder and iterate over all the images
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import os.path as osp
import cv2 as cv
import numpy as np

# set the directory
VIDEO_ID = '20200807015315'
data_dir = 'out/TEST'
video_dir = osp.join(data_dir, VIDEO_ID, 'color')

# for every image in the color folder in alphabetical order
for mask_frame in sorted(os.listdir(video_dir)):
    if not mask_frame.endswith('_raw.png'):
        continue

    # load the mask
    mask_path = osp.join(video_dir, mask_frame)
    mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)

    # separate the mask into individual layers depending on it's value
    mask = separate_layers(mask)

    # extract blobs from each layer
    blobs = [extract_blobs(layer) for layer in mask]
    
    num_blobs = 0
    for layer in blobs:
        num_blobs += len(layer)
    print(f"Frame {mask_frame} has {num_blobs} blobs in total")

    # visualize the blobs as a red bounding box on top of the image
    colored_og_maks_path = osp.join(video_dir, mask_frame.replace('_raw.png', '_rgb.jpg'))
    colored_og_mask = cv.imread(colored_og_maks_path)
    for layer in blobs:
        for blob in layer:
            print(blob['bounding_box'])
            x1, y1, w, h = blob['bounding_box']
            x2, y2 = x1 + w, y1 + h
            cv.rectangle(colored_og_mask, (x1, y1), (x2, y2), (0, 0, 255), 1)
            # also draw the centroid as a red dot
            cx, cy = blob['centroid']
            cv.circle(colored_og_mask, (cx, cy), 2, (0, 0, 255), -1)

    # save the image
    out_path = osp.join(video_dir, mask_frame.replace('_raw', 'BLOBBED'))
    print(cv.imwrite(out_path, colored_og_mask))
    print(f"Saved {out_path}")


Frame 000001_raw.png has 0 blobs in total
True
Saved out/TEST/20200807015315/color/000001BLOBBED.png
Frame 000011_raw.png has 0 blobs in total
True
Saved out/TEST/20200807015315/color/000011BLOBBED.png
Frame 000021_raw.png has 3 blobs in total
(591, 186, 40, 41)
(366, 388, 27, 23)
(798, 122, 26, 28)
True
Saved out/TEST/20200807015315/color/000021BLOBBED.png
Frame 000031_raw.png has 9 blobs in total
(473, 437, 38, 43)
(521, 426, 49, 54)
(486, 400, 44, 25)
(588, 343, 67, 53)
(582, 307, 23, 34)
(617, 304, 20, 24)
(298, 303, 83, 107)
(254, 253, 37, 61)
(656, 221, 46, 58)
True
Saved out/TEST/20200807015315/color/000031BLOBBED.png
Frame 000041_raw.png has 5 blobs in total
(481, 409, 57, 71)
(565, 304, 69, 88)
(243, 262, 103, 128)
(820, 243, 28, 30)
(638, 221, 49, 55)
True
Saved out/TEST/20200807015315/color/000041BLOBBED.png
Frame 000051_raw.png has 5 blobs in total
(444, 390, 94, 90)
(548, 300, 67, 88)
(238, 267, 88, 107)
(796, 251, 52, 33)
(624, 226, 41, 50)
True
Saved out/TEST/20200807015

In [3]:
import cv2 as cv
import seaborn as sns
import os.path as osp
from src.sort.sort import Sort

# VIDEO_ID = 'bag_20210811222439'
# VIDEO_ID = 'bag_20210802015851'
VIDEO_ID = '20200807015315'
data_dir = 'out/TEST'
video_dir = osp.join(data_dir, VIDEO_ID, 'color')

tracker = Sort(max_age=5, min_hits=2, iou_threshold=0.0)

# generate a list of 20 colors (intense colors)
palette = sns.color_palette('hsv', 20)
colors = [(int(255 * r), int(255 * g), int(255 * b)) for r, g, b in palette]

for mask_id, mask_frame in enumerate(sorted(os.listdir(video_dir))):
    if not mask_frame.endswith('_raw.png'):
        continue
    print('extracting blobs from', mask_frame)

    # load the mask
    mask_path = osp.join(video_dir, mask_frame)
    mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)

    # separate the mask into individual layers depending on it's value
    mask = separate_layers(mask)
    print(f"mask shape: {mask.shape}")

    # extract blobs from each layer
    all_bbs = []
    for layer in mask:
        layer_blobs = extract_blobs(layer, merge_distance=1)
        layer_bbs = [blob['bounding_box'] for blob in layer_blobs]
        all_bbs += layer_bbs
    # parse x, y, w, h into x1, y1, x2, y2
    all_bbs = [[x, y, x + w, y + h] for (x, y, w, h) in all_bbs]

    # update the tracker
    tracked_objects = tracker.update(np.asarray(all_bbs))


    rgb_og__mask_path = osp.join(video_dir, mask_frame.replace('_raw.png', '_rgb.jpg'))
    rgb_og_mask = cv.imread(rgb_og__mask_path)
    for obj in tracked_objects:
        x1, y1, x2, y2, track_id = obj
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Convert to integers
        # assign a color depending on the track id
        color = colors[int(track_id)]
        # Draw bounding box
        cv.rectangle(rgb_og_mask, (x1, y1), (x2, y2), color, 2)
        # Draw track ID
        cv.putText(rgb_og_mask, f"ID: {int(track_id)}", (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 1.2, color, 2)

    # Save the annotated frame
    out_path = osp.join(video_dir, mask_frame.replace('_raw', 'IDS'))
    print(cv.imwrite(out_path, rgb_og_mask))
    print(f"Saved {out_path}")
    
    

extracting blobs from 000001_raw.png
mask shape: (4, 480, 848)
True
Saved out/TEST/20200807015315/color/000001IDS.png
extracting blobs from 000011_raw.png
mask shape: (4, 480, 848)
True
Saved out/TEST/20200807015315/color/000011IDS.png
extracting blobs from 000021_raw.png
mask shape: (4, 480, 848)
True
Saved out/TEST/20200807015315/color/000021IDS.png
extracting blobs from 000031_raw.png
mask shape: (4, 480, 848)
True
Saved out/TEST/20200807015315/color/000031IDS.png
extracting blobs from 000041_raw.png
mask shape: (4, 480, 848)
True
Saved out/TEST/20200807015315/color/000041IDS.png
extracting blobs from 000051_raw.png
mask shape: (4, 480, 848)
True
Saved out/TEST/20200807015315/color/000051IDS.png
extracting blobs from 000061_raw.png
mask shape: (4, 480, 848)
True
Saved out/TEST/20200807015315/color/000061IDS.png
extracting blobs from 000071_raw.png
mask shape: (4, 480, 848)
True
Saved out/TEST/20200807015315/color/000071IDS.png
extracting blobs from 000081_raw.png
mask shape: (4, 480

In [13]:
import os
import os.path as osp
import cv2 as cv
import numpy as np

VIDEO_ID = '20200807015315'
data_dir = 'out/TEST'
video_dir = osp.join(data_dir, VIDEO_ID, 'color')

stitched_img = None
rows = []

# Sort the frame files and process
frame_files = [f for f in sorted(os.listdir(video_dir)) if f.endswith('IDS.png')]

for i, id_frame in enumerate(frame_files):
    # if i % 2 != 0:  # only every 2nd frame
    #     continue

    id_path = osp.join(video_dir, id_frame)
    id_img = cv.imread(id_path)

    # add the frame number on the top right corner (black background and white text)
    frame_num = id_frame[3:6]
    cv.rectangle(id_img, (0, 0), (200, 40), (0, 0, 0), -1)
    cv.putText(id_img, f"Frame {frame_num}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Determine the row index
    row_num = i // 6

    # If a new row, initialize it; otherwise, concatenate horizontally
    if len(rows) <= row_num:
        rows.append(id_img)  # Start a new row
    else:
        rows[row_num] = np.hstack((rows[row_num], id_img))  # Add to the existing row

# Combine all rows vertically to create the stitched image
# check if all rows have the same number of columns, if not fill with white
max_cols = max(row.shape[1] for row in rows)
for i, row in enumerate(rows):
    if row.shape[1] < max_cols:
        rows[i] = np.hstack((row, 255 * np.ones((row.shape[0], max_cols - row.shape[1], 3), dtype=np.uint8)))


stitched_img = np.vstack(rows)

# Save the stitched image
output_path = osp.join(video_dir, 'stitched.png')
cv.imwrite(output_path, stitched_img)
print(f"Stitched image saved to {output_path}")


Stitched image saved to out/TEST/20200807015315/color/stitched.png
